In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
num_epochs = 80
batch_size=100
learning_rate=0.01

In [10]:
transform = transforms.Compose([
    # 对图像进行填充，比如原来32x32,四周填充4个就是40x40
    transforms.Pad(4),
    # 对图像进行随机水平翻转
    transforms.RandomHorizontalFlip(),
    # 对图像进行随机裁剪至32x32的大小
    transforms.RandomCrop(32),
    # 转化为张量
    transforms.ToTensor()
])

In [11]:
train_dataset = torchvision.datasets.CIFAR10(root="./data",
                                             download=True,
                                             train=True,
                                            transform = transform
                                            )

Files already downloaded and verified


In [12]:
test_dataset = torchvision.datasets.CIFAR10(root="./data",
                                           train=False,
                                           transform=transforms.ToTensor())

In [13]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,shuffle=True,batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,shuffle=False,batch_size=batch_size)

In [14]:
# 定义了3x3的卷积
def conv3x3(in_channels,out_channels,stride=1):
    return nn.Conv2d(in_channels,out_channels,kernel_size=3,stride = stride,padding=1,bias=False)

In [28]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels , out_channels,stride=1,downsample=None):
        # super方法代表希望继承父类nn.Module的方法
        super(ResidualBlock,self).__init__()
        self.conv1=conv3x3(in_channels,out_channels,stride)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels,out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self,x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out +=residual
        out = self.relu(out)
        return out

In [29]:
class Resnet(nn.Module):
    def __init__(self,block,layers,num_classes=10):
        super(Resnet,self).__init__()
        self.in_channels =16
        self.conv=conv3x3(3,16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block=block, 
                                      out_channels=16,
                                      blocksNumOfEachBlock=layers[0])
        self.layer2 = self.make_layer(block, 
                                      out_channels=32, 
                                      blocksNumOfEachBlock=layers[1],
                                      stride= 2)
        self.layer3 = self.make_layer(block, 
                                      out_channels=64, 
                                      blocksNumOfEachBlock=layers[2],
                                      stride= 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self,block,out_channels,blocksNumOfEachBlock,stride=1):
        downsample=None
        if(stride!=1) or (self.in_channels!=out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels,out_channels,stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        layers=[]
        layers.append(block(self.in_channels,out_channels,stride,downsample))
        self.in_channels=out_channels
        for i in range(1,blocksNumOfEachBlock):
            layers.append(block(out_channels,out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [31]:
model = Resnet(ResidualBlock,[2,2,2]).to(device)

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [33]:
def update_lr(optimizer,lr):
    for param_group in optimizer.param_groups:
        param_group['lr']=lr

In [34]:
total_step = len(train_loader)
curr_lr = learning_rate

In [35]:
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        loss= criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    if(epoch+1)%20 ==0:
        curr_lr/=3
        update_lr(optimizer,curr_lr)
    

Epoch [1/80], Step [100/500] Loss: 1.9761
Epoch [1/80], Step [200/500] Loss: 1.4832
Epoch [1/80], Step [300/500] Loss: 1.4562
Epoch [1/80], Step [400/500] Loss: 1.4500
Epoch [1/80], Step [500/500] Loss: 1.2846
Epoch [2/80], Step [100/500] Loss: 1.1666
Epoch [2/80], Step [200/500] Loss: 1.2283
Epoch [2/80], Step [300/500] Loss: 1.1652
Epoch [2/80], Step [400/500] Loss: 1.2397
Epoch [2/80], Step [500/500] Loss: 0.9170
Epoch [3/80], Step [100/500] Loss: 0.9886
Epoch [3/80], Step [200/500] Loss: 0.8639
Epoch [3/80], Step [300/500] Loss: 0.8064
Epoch [3/80], Step [400/500] Loss: 0.9036
Epoch [3/80], Step [500/500] Loss: 0.8662
Epoch [4/80], Step [100/500] Loss: 0.6300
Epoch [4/80], Step [200/500] Loss: 0.8094
Epoch [4/80], Step [300/500] Loss: 0.7108
Epoch [4/80], Step [400/500] Loss: 0.6659
Epoch [4/80], Step [500/500] Loss: 0.9315
Epoch [5/80], Step [100/500] Loss: 0.8467
Epoch [5/80], Step [200/500] Loss: 0.6671
Epoch [5/80], Step [300/500] Loss: 0.6894
Epoch [5/80], Step [400/500] Loss:

Epoch [39/80], Step [300/500] Loss: 0.2680
Epoch [39/80], Step [400/500] Loss: 0.1381
Epoch [39/80], Step [500/500] Loss: 0.1507
Epoch [40/80], Step [100/500] Loss: 0.2413
Epoch [40/80], Step [200/500] Loss: 0.1426
Epoch [40/80], Step [300/500] Loss: 0.1609
Epoch [40/80], Step [400/500] Loss: 0.1237
Epoch [40/80], Step [500/500] Loss: 0.1179
Epoch [41/80], Step [100/500] Loss: 0.1300
Epoch [41/80], Step [200/500] Loss: 0.1604
Epoch [41/80], Step [300/500] Loss: 0.1424
Epoch [41/80], Step [400/500] Loss: 0.1363
Epoch [41/80], Step [500/500] Loss: 0.1455
Epoch [42/80], Step [100/500] Loss: 0.0777
Epoch [42/80], Step [200/500] Loss: 0.0768
Epoch [42/80], Step [300/500] Loss: 0.2304
Epoch [42/80], Step [400/500] Loss: 0.3064
Epoch [42/80], Step [500/500] Loss: 0.2453
Epoch [43/80], Step [100/500] Loss: 0.1348
Epoch [43/80], Step [200/500] Loss: 0.0984
Epoch [43/80], Step [300/500] Loss: 0.1936
Epoch [43/80], Step [400/500] Loss: 0.0966
Epoch [43/80], Step [500/500] Loss: 0.1868
Epoch [44/8

Epoch [77/80], Step [400/500] Loss: 0.1093
Epoch [77/80], Step [500/500] Loss: 0.0342
Epoch [78/80], Step [100/500] Loss: 0.0384
Epoch [78/80], Step [200/500] Loss: 0.0684
Epoch [78/80], Step [300/500] Loss: 0.1044
Epoch [78/80], Step [400/500] Loss: 0.0250
Epoch [78/80], Step [500/500] Loss: 0.0632
Epoch [79/80], Step [100/500] Loss: 0.0541
Epoch [79/80], Step [200/500] Loss: 0.1539
Epoch [79/80], Step [300/500] Loss: 0.0721
Epoch [79/80], Step [400/500] Loss: 0.0514
Epoch [79/80], Step [500/500] Loss: 0.1285
Epoch [80/80], Step [100/500] Loss: 0.0992
Epoch [80/80], Step [200/500] Loss: 0.1601
Epoch [80/80], Step [300/500] Loss: 0.1016
Epoch [80/80], Step [400/500] Loss: 0.0304
Epoch [80/80], Step [500/500] Loss: 0.0847


In [36]:
model.eval()

Resnet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

In [37]:
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images= images.to(device)
        labels = images.to(device)
        outputs = model(images)
        _,pred= torch.max(outputs,1)
        total+=labels.size(0)
        correct+=(pred == labels).sum()
    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


RuntimeError: The size of tensor a (100) must match the size of tensor b (32) at non-singleton dimension 3

In [ ]:
torch.save(model.state_dict(), 'resnet.ckpt')
